# Import libs

In [1]:
# Import some stuff
import os
from dotenv import load_dotenv
from pathlib import Path
from pymongo import MongoClient
import matplotlib.pyplot as plt
import pandas as pd
import ast
import datetime
import seaborn as sns
import json
import numpy as np

import matplotlib as mpl

from datetime import datetime
from dateutil.parser import parse
import pandas as pd


from pandas.io.json import json_normalize
#from wordcloud import WordCloud, STOPWORDS


In [2]:
# specifies number of rows to show
pd.set_option('display.max_rows', 300) 

# specifies default number format to 4 decimal places
pd.options.display.float_format = '{:40,.4f}'.format 

# specifies that graphs should use ggplot styling
plt.style.use('ggplot') 



plt.rcParams['figure.figsize'] = 50, 30

%matplotlib inline

In [3]:
# load .env variable
base_dir = 'D:\Documentos\GitHub\CityBlender'
dotenv_file = os.path.join(base_dir, '.env')


if os.path.isfile(dotenv_file):
    load_dotenv(dotenv_file, verbose=True)
    
db_uri = os.getenv('DB_URI')

In [ ]:
# create a new MongoDB Client
db_client = MongoClient(str(db_uri))

# If it's ok it should return a 1.0
db_client.london.command("serverStatus")['ok']

In [ ]:
# choose a database to connect to (.london)
db_london = db_client.london

# choose a collection
db_london_events = db_london['events']
db_london_artist = db_london['artists']

In [ ]:
# lookup all the documents in a collection
db_london_events.find()

## EOD

In [ ]:
events = pd.DataFrame(list(db_london_events.find({})))

In [ ]:
events.columns

### For instance in the first row there are 3 events

In [ ]:
# Put this in the date fomrat
events['date'] = [(datetime.strptime(events['date'][x], '%Y-%m-%d')) for x in range(len(events['date']))]

# Convert the dates into day name
events['date_name'] = [(events['date'][x].strftime("%A")) for x in range(len(events['date']))]

# Lat & Long
events['lat'] = [(events['location'][x]['lat']) for x in range(len(events['location']))]
events['lng'] = [(events['location'][x]['lng']) for x in range(len(events['location']))]

## Spotify

In [ ]:
type(events['spotify'][0])

In [ ]:
for key in events['spotify'][0]:
  print(key)

In [ ]:
def plot_a_scatter(db = events, x = 'tempo_median', y = 'acousticness_median'):

    for_plot = pd.DataFrame()
    list_of_x = []
    list_of_y = []
    
    for i in range(len(db['spotify'])):
        if type(db['spotify'][i]) != float:
            list_of_x.extend(db['spotify'][i][x])

    for i in range(len(db['spotify'])):
        if type(db['spotify'][i]) != float:
            list_of_y.extend(db['spotify'][i][y])
            
    for_plot[x] = list_of_x
    for_plot[y] = list_of_y
    
    plt.scatter(for_plot[x], for_plot[y])

In [ ]:
plot_a_scatter(db = events, x = 'tempo', y = 'acousticness')

In [ ]:
plot_a_scatter(db = events, x = 'liveness', y = 'energy')

In [ ]:
plot_a_scatter(db = events, x = 'danceability', y = 'energy')

## Genere - What to do with missing values

In [ ]:
list_of_generes = []

for i in range(len(events['spotify'])):
    if type(events['spotify'][i]) != float:
        list_of_generes.extend(events['spotify'][i]['genres'])
    


In [ ]:
events['spotify'][2]['genres']

## Word cloud

In [ ]:
len(list_of_generes)

In [ ]:
data = list_of_generes

In [ ]:
from wordcloud import WordCloud, STOPWORDS

mpl.rcParams['figure.figsize']=(25,25)    #(6.0,4.0)
mpl.rcParams['font.size']=12                #10 
mpl.rcParams['savefig.dpi']=1200             #72 
mpl.rcParams['figure.subplot.bottom']=.1 

seed = 320
stopwords = set(STOPWORDS)
data = list_of_generes

wordcloud = WordCloud(width=600, height=200,
                          background_color='grey',
                          stopwords=stopwords,
                          max_words=100,
                          max_font_size=60, 
                          random_state=42
                         ).generate(str((data)))



plt.figure( figsize=(20,10), facecolor='k')
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

In [ ]:
pd.Series(data).value_counts().plot('bar')

## Spotify vars

In [ ]:
def plot_distributions(db = events, var = 'acousticness_median'):
    list_of_var = []

    for i in range(len(db['spotify'])):
        if type(db['spotify'][i]) != float:
            list_of_var.append(db['spotify'][i][var])
    
    
    plt.hist(list_of_var)

In [ ]:
plot_distributions(events, var = 'tempo_min')

In [ ]:
list_of_acousticness = []

for i in range(len(events['spotify'])):
    if type(events['spotify'][i]) != float:
        list_of_acousticness.append(events['spotify'][i]['acousticness_median'])
    

In [ ]:
plt.hist(list_of_acousticness)

In [ ]:
list_of_tempo = []
list_of_dance =[]

db = events

for i in range(len(db['spotify'])):
        if type(db['spotify'][i]) != float:
            list_of_tempo.append(db['spotify'][i]['energy_median'])
            
for i in range(len(db['spotify'])):
        if type(db['spotify'][i]) != float:
            list_of_dance.append(db['spotify'][i]['popularity_median'])            

In [ ]:
plt.scatter(list_of_tempo, list_of_dance)


plt.tick_params(labelsize=28)

## City Level

## Bourogh Level

## Gigs data